# Importing necessary files

In [1]:
import pandas as pd
import numpy as np
import os
SEED = 42
import random as rn
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

# Preprocessing the data

In [2]:
data=os.listdir(r"C:\Users\Nancy Meshram\Downloads\required_duration_logits\required_duration_logits")

In [5]:
len(data)

7962

In [6]:
meta=pd.read_csv(r"C:\Users\Nancy Meshram\Downloads\meta2.csv")
meta=meta.drop(columns=['Unnamed: 0','Unnamed: 0.1'])
y=meta.iloc[:,1:].values

In [7]:
y

array([[1],
       [1],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int64)

In [8]:
from sklearn.model_selection import train_test_split
y_train,y_test_val=train_test_split(y,train_size=0.8,random_state=42)
y_val,y_test=train_test_split(y_test_val,test_size=0.5,random_state=42)

In [9]:
print(len(y_train),len(y_val),len(y_test))

6369 796 797


In [17]:
import os
test_npy_files=os.listdir(r"C:\Users\Nancy Meshram\Downloads\test\test")
train_npy_files=os.listdir(r"C:\Users\Nancy Meshram\Downloads\train\train")
val_npy_files=os.listdir(r"C:\Users\Nancy Meshram\Downloads\val\val")

In [18]:
print(len(test_npy_files),len(train_npy_files),len(val_npy_files))

797 6369 796


In [19]:
test_size=[]
for i in test_npy_files:
  xx=np.load(r"C:\Users\Nancy Meshram\Downloads\test\test"+'\\'+i).shape
  test_size.append(xx)

In [20]:
val_size=[]
for i in val_npy_files:
  xx=np.load(r"C:\Users\Nancy Meshram\Downloads\val\val"+'\\'+i).shape
  val_size.append(xx)

In [21]:
train_size=[]
for i in train_npy_files:
  xx=np.load(r"C:\Users\Nancy Meshram\Downloads\train\train"+'\\'+i).shape
  train_size.append(xx)

In [22]:
print(max(train_size),max(test_size),max(val_size))

(2072, 123) (1509, 123) (1443, 123)


In [23]:
############### Creating numpy arrays for train, test and validation data ##################

train=np.zeros((6369,2072,123),dtype='float32')
test=np.zeros((797,2072,123),dtype='float32')
val=np.zeros((796,2072,123),dtype='float32')

In [24]:
########### Test Data ###########

for i, j in enumerate(test_npy_files):
  xx=np.load(r"C:\Users\Nancy Meshram\Downloads\test\test"+'\\'+j).shape[0]
  test[i,:xx,:]=np.load(r"C:\Users\Nancy Meshram\Downloads\test\test"+'\\'+j)

In [25]:
############ Validation Data ###############

for i, j in enumerate(val_npy_files):
  xx=np.load(r"C:\Users\Nancy Meshram\Downloads\val\val"+'\\'+j).shape[0]
  val[i,:xx,:]=np.load(r"C:\Users\Nancy Meshram\Downloads\val\val"+'\\'+j)

In [26]:
############ Train Data ################

for i, j in enumerate(train_npy_files):
  xx=np.load(r"C:\Users\Nancy Meshram\Downloads\train\train"+'\\'+j).shape[0]
  train[i,:xx,:]=np.load(r"C:\Users\Nancy Meshram\Downloads\train\train"+'\\'+j)

In [27]:
print(train.shape,test.shape,val.shape)

(6369, 2072, 123) (797, 2072, 123) (796, 2072, 123)


In [28]:
n_timesteps, n_features, n_outputs = train.shape[1], train.shape[2], y_train.shape[1]

In [29]:
print(n_timesteps, n_features, n_outputs)

2072 123 1


# Balancing train,test and val data

In [30]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

In [31]:
train_data2d = train.reshape(train.shape[0], -1)
train_data2d.shape

(6369, 254856)

In [32]:
undersample = RandomUnderSampler(sampling_strategy='majority')
X_train_set, y_train_set = undersample.fit_resample(train_data2d, y_train)
print(X_train_set.shape)
print(y_train_set.shape)

(3810, 254856)
(3810,)


In [33]:
y_train_set = np.reshape(y_train_set, (y_train_set.shape[0], 1))
print(y_train_set.shape)

(3810, 1)


In [34]:
train_set = X_train_set.reshape(-1, train.shape[1], train.shape[2])
train_set.shape

(3810, 2072, 123)

In [35]:
Counter(y_train_set.ravel())

Counter({0: 1905, 1: 1905})

In [36]:
test_data2d = test.reshape(test.shape[0], -1)
test_data2d.shape

(797, 254856)

In [37]:
X_test_set, y_test_set = undersample.fit_resample(test_data2d, y_test)
print(X_test_set.shape,y_test_set.shape)

(520, 254856) (520,)


In [38]:
y_test_set = np.reshape(y_test_set, (y_test_set.shape[0], 1))
test_set = X_test_set.reshape(-1, test.shape[1], test.shape[2])
test_set.shape

(520, 2072, 123)

In [43]:
val_data2d = val.reshape(val.shape[0], -1)
val_data2d.shape

(796, 254856)

In [44]:
X_val_set, y_val_set = undersample.fit_resample(val_data2d, y_val)
print(X_val_set.shape,y_val_set.shape)

(514, 254856) (514,)


In [45]:
y_val_set = np.reshape(y_val_set, (y_val_set.shape[0], 1))
val_set = X_val_set.reshape(-1, val.shape[1], val.shape[2])
val_set.shape

(514, 2072, 123)

In [46]:
Counter(y_val_set.ravel())

Counter({0: 257, 1: 257})

# Building the Model

In [47]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import BatchNormalization
import tensorflow as tf

In [305]:
model=Sequential()
model.add(Conv1D(filters=32, kernel_size=10, activation='relu',input_shape=(2072,123))) ##### LAYER 1 #####
model.add(Conv1D(filters=64, kernel_size=10, activation='relu')) ##### LAYER 2 #####
model.add(Conv1D(filters=128, kernel_size=10, activation='relu')) ##### LAYER 3 #####
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(Dense(128, activation='relu')) ##### FULLY CONNECTED LAYER #####
model.add(Dense(n_outputs, activation='sigmoid')) ##### OUTPUT LAYER #####

In [306]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.0005)
# adam=tf.keras.optimizers.Adam(learning_rate=0.0005)
# tf.keras.optimizers.RMSprop(learning_rate=0.001)

In [307]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer=sgd,metrics=['accuracy'])

In [308]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_25 (Conv1D)          (None, 2063, 32)          39392     
                                                                 
 conv1d_26 (Conv1D)          (None, 2054, 64)          20544     
                                                                 
 batch_normalization_7 (Batc  (None, 2054, 64)         256       
 hNormalization)                                                 
                                                                 
 dropout_8 (Dropout)         (None, 2054, 64)          0         
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 684, 64)          0         
 1D)                                                             
                                                                 
 flatten_8 (Flatten)         (None, 43776)            

In [309]:
model.fit(train_set,y_train_set,batch_size=4,epochs=5,verbose=True,validation_data=(val_set,y_val_set),shuffle = True)

Epoch 1/5
953/953 [==============================] - 64s 66ms/step - loss: 0.7270 - accuracy: 0.5016 - val_loss: 0.6958 - val_accuracy: 0.4961
Epoch 2/5
953/953 [==============================] - 66s 69ms/step - loss: 0.6849 - accuracy: 0.5533 - val_loss: 0.6957 - val_accuracy: 0.4767
Epoch 3/5
953/953 [==============================] - 71s 74ms/step - loss: 0.6626 - accuracy: 0.6055 - val_loss: 0.6998 - val_accuracy: 0.5175
Epoch 4/5
953/953 [==============================] - 73s 77ms/step - loss: 0.6261 - accuracy: 0.6493 - val_loss: 0.7250 - val_accuracy: 0.4903
Epoch 5/5
953/953 [==============================] - 62s 65ms/step - loss: 0.5701 - accuracy: 0.7097 - val_loss: 0.7361 - val_accuracy: 0.4883


In [310]:
_, accuracy = model.evaluate(train_set, y_train_set, batch_size=4, verbose=True) ###Train Accuracy###

953/953 [==============================] - 10s 11ms/step - loss: 0.5133 - accuracy: 0.7942


In [311]:
_, accuracy = model.evaluate(val_set, y_val_set, batch_size=4, verbose=True) ###Val Accuracy###

129/129 [==============================] - 2s 13ms/step - loss: 0.7361 - accuracy: 0.4883


In [312]:
_, accuracy = model.evaluate(test_set, y_test_set, batch_size=4, verbose=True) ###Test Accuracy###

130/130 [==============================] - 2s 15ms/step - loss: 0.7354 - accuracy: 0.5288


# Train

In [313]:
y_pred_train=model.predict(train_set)

In [314]:
y_pred_train

array([[0.18264008],
       [0.43466854],
       [0.17138591],
       ...,
       [0.5361414 ],
       [0.45852092],
       [0.5915519 ]], dtype=float32)

In [315]:
print(max(y_pred_train),min(y_pred_train))

[0.8647896] [0.02319247]


In [316]:
y_pred_train_1 = np.where(y_pred_train < 0.5, 0, 1)

In [317]:
y_pred_train_1

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [1]])

In [318]:
from sklearn.metrics import classification_report
print(classification_report(y_train_set, y_pred_train_1))

              precision    recall  f1-score   support

           0       0.72      0.97      0.82      1905
           1       0.95      0.62      0.75      1905

    accuracy                           0.79      3810
   macro avg       0.84      0.79      0.79      3810
weighted avg       0.84      0.79      0.79      3810



In [319]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train_set, y_pred_train_1) 
cm

array([[1847,   58],
       [ 726, 1179]], dtype=int64)

In [320]:
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
print(f1_score(y_train_set, y_pred_train_1, average='macro'))
print(precision_score(y_train_set, y_pred_train_1, average='macro'))
print(recall_score(y_train_set, y_pred_train_1, average='macro'))
print(accuracy_score(y_train_set, y_pred_train_1))

0.787699621517631
0.8354757334812952
0.7942257217847769
0.7942257217847769


# Validation

In [321]:
y_pred_val=model.predict(val_set)

In [322]:
y_pred_val

array([[0.22536433],
       [0.34483117],
       [0.47839713],
       [0.30419755],
       [0.39322984],
       [0.547653  ],
       [0.35941347],
       [0.26138672],
       [0.43159524],
       [0.5416293 ],
       [0.40077803],
       [0.46145937],
       [0.36198968],
       [0.43136567],
       [0.252706  ],
       [0.1277487 ],
       [0.52896863],
       [0.50147945],
       [0.43844175],
       [0.6069415 ],
       [0.21214283],
       [0.4078213 ],
       [0.35808071],
       [0.4756935 ],
       [0.22104096],
       [0.32812816],
       [0.16411448],
       [0.45190766],
       [0.26929057],
       [0.47218162],
       [0.32484055],
       [0.3941881 ],
       [0.651022  ],
       [0.23029268],
       [0.2623586 ],
       [0.30534714],
       [0.5908833 ],
       [0.35969445],
       [0.34477645],
       [0.49078968],
       [0.54810584],
       [0.2935369 ],
       [0.16097885],
       [0.30001557],
       [0.4029649 ],
       [0.5263128 ],
       [0.37884873],
       [0.346

In [323]:
print(max(y_pred_val),min(y_pred_val))

[0.77233773] [0.08759335]


In [324]:
y_pred_val_1 = np.where(y_pred_val < 0.5, 0, 1)

In [325]:
y_pred_val_1

array([[0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
    

In [326]:
from sklearn.metrics import classification_report
print(classification_report(y_val_set, y_pred_val_1))

              precision    recall  f1-score   support

           0       0.49      0.74      0.59       257
           1       0.48      0.24      0.32       257

    accuracy                           0.49       514
   macro avg       0.48      0.49      0.45       514
weighted avg       0.48      0.49      0.45       514



In [327]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val_set, y_pred_val_1) 
cm

array([[190,  67],
       [196,  61]], dtype=int64)

In [328]:
print(f1_score(y_val_set, y_pred_val_1, average='macro'))
print(precision_score(y_val_set, y_pred_val_1, average='macro'))
print(recall_score(y_val_set, y_pred_val_1, average='macro'))
print(accuracy_score(y_val_set, y_pred_val_1))

0.4539314495768617
0.4843952396373057
0.4883268482490273
0.4883268482490272


# Test

In [329]:
y_pred_test=model.predict(test_set)

In [330]:
y_pred_test

array([[0.31490314],
       [0.42468247],
       [0.22051844],
       [0.63721675],
       [0.386253  ],
       [0.34599188],
       [0.5828507 ],
       [0.32987848],
       [0.4946047 ],
       [0.4546256 ],
       [0.47736096],
       [0.3108245 ],
       [0.2930845 ],
       [0.5897221 ],
       [0.20234561],
       [0.4202724 ],
       [0.24957553],
       [0.24477151],
       [0.28304976],
       [0.42794907],
       [0.39831093],
       [0.35252887],
       [0.56457216],
       [0.25586215],
       [0.44947234],
       [0.39837188],
       [0.43758664],
       [0.38127843],
       [0.36283463],
       [0.37485772],
       [0.35165137],
       [0.33413345],
       [0.185469  ],
       [0.35253486],
       [0.41580313],
       [0.27242672],
       [0.15857327],
       [0.32671094],
       [0.24001911],
       [0.38991886],
       [0.47072107],
       [0.5749969 ],
       [0.54413044],
       [0.56148386],
       [0.2789917 ],
       [0.15266126],
       [0.43269166],
       [0.525

In [331]:
print(min(y_pred_test),max(y_pred_test))

[0.1286807] [0.73107445]


In [332]:
y_pred_test_1 = np.where(y_pred_test < 0.5, 0, 1)

In [333]:
y_pred_test_1

array([[0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
    

In [334]:
from sklearn.metrics import classification_report
print(classification_report(y_test_set, y_pred_test_1))

              precision    recall  f1-score   support

           0       0.52      0.79      0.63       260
           1       0.56      0.27      0.36       260

    accuracy                           0.53       520
   macro avg       0.54      0.53      0.49       520
weighted avg       0.54      0.53      0.49       520



In [335]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_set, y_pred_test_1) 
cm

array([[205,  55],
       [190,  70]], dtype=int64)

In [336]:
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
print(f1_score(y_test_set, y_pred_test_1, average='macro'))
print(precision_score(y_test_set, y_pred_test_1, average='macro'))
print(recall_score(y_test_set, y_pred_test_1, average='macro'))
print(accuracy_score(y_test_set, y_pred_test_1))

0.49479528105482307
0.539493670886076
0.5288461538461539
0.5288461538461539
